# Top 100 Movies of All Time Analysis
## Data Processing

In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('../data/raw/Top_100_Movies.csv')
df.head()

,Unnamed: 0,rank,title,description,genre,rating,id,year,imdbid,imdb_link,image
0,0,1,The Shawshank Redemption,Two imprisoned men bond over a number of years...,['Drama'],9.3,top1,1994,tt0111161,https://www.imdb.com/title/tt0111161,https://m.media-amazon.com/images/M/MV5BMDFkYT...
1,1,2,The Godfather,The aging patriarch of an organized crime dyna...,"['Crime', 'Drama']",9.2,top2,1972,tt0068646,https://www.imdb.com/title/tt0068646,https://m.media-amazon.com/images/M/MV5BM2MyNj...
2,2,3,The Dark Knight,When the menace known as the Joker wreaks havo...,"['Action', 'Crime', 'Drama']",9.0,top3,2008,tt0468569,https://www.imdb.com/title/tt0468569,https://m.media-amazon.com/images/M/MV5BMTMxNT...
3,3,4,The Godfather Part II,The early life and career of Vito Corleone in ...,"['Crime', 'Drama']",9.0,top4,1974,tt0071562,https://www.imdb.com/title/tt0071562,https://m.media-amazon.com/images/M/MV5BMWMwMG...
4,4,5,12 Angry Men,The jury in a New York City murder trial is fr...,"['Crime', 'Drama']",9.0,top5,1957,tt0050083,https://www.imdb.com/title/tt0050083,https://m.media-amazon.com/images/M/MV5BMWU4N2...


In [27]:
# store imdb links in list for extra data extraction

imdb_links = df['imdb_link'].tolist()

In [28]:
# drop unnecessary columns and rename rating to imdb_rating
df.drop(columns=['Unnamed: 0', 'imdbid', 'image', 'imdb_link', 'description', 'id'], inplace=True)

df.rename(columns={'rating': 'imdb_rating'}, inplace=True)

df.head()

,rank,title,genre,imdb_rating,year
0,1,The Shawshank Redemption,['Drama'],9.3,1994
1,2,The Godfather,"['Crime', 'Drama']",9.2,1972
2,3,The Dark Knight,"['Action', 'Crime', 'Drama']",9.0,2008
3,4,The Godfather Part II,"['Crime', 'Drama']",9.0,1974
4,5,12 Angry Men,"['Crime', 'Drama']",9.0,1957


In [29]:
from bs4 import BeautifulSoup
import requests

In [30]:
# headers to allow scraping
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# initialize lists for ratings and times
rating_list = []
time_list = []

for link in imdb_links:
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # check to see if rating exists on IMDB and add to list
    try:
        rating = soup.select_one('div.iPPbjm li:nth-of-type(2) a').text
    except AttributeError:
        rating = 'Not Rated'
    rating_list.append(rating)
    
    # check to see if the time is the second or third list item 
    try:
        total_time = soup.select_one('div.iPPbjm li:nth-of-type(3)').text.replace('h', '').replace('m', '').split(' ')
    except AttributeError:
        total_time = soup.select_one('div.iPPbjm li:nth-of-type(2)').text.replace('h', '').replace('m', '').split(' ')
        
    # check to see if the time contains hours and minutes or just hours and add to list
    if len(total_time) == 2:
        time = int(total_time[0]) * 60 + int(total_time[1])
    else:
        time = int(total_time[0]) * 60
    time_list.append(time)

In [31]:
# add ratings and runtime to df

df['rating'] = rating_list
df['run_time'] = time_list

df.head()

,rank,title,genre,imdb_rating,year,rating,run_time
0,1,The Shawshank Redemption,['Drama'],9.3,1994,R,142
1,2,The Godfather,"['Crime', 'Drama']",9.2,1972,R,175
2,3,The Dark Knight,"['Action', 'Crime', 'Drama']",9.0,2008,PG-13,152
3,4,The Godfather Part II,"['Crime', 'Drama']",9.0,1974,R,202
4,5,12 Angry Men,"['Crime', 'Drama']",9.0,1957,Approved,96


In [32]:
# split the end of the year to make a decades column

df_split = df['year'].astype(str).str.split('', expand=True)

df_split[4] = '0'

df['decade'] = df_split[3] + df_split[4]

df.head(5)

,rank,title,genre,imdb_rating,year,rating,run_time,decade
0,1,The Shawshank Redemption,['Drama'],9.3,1994,R,142,90
1,2,The Godfather,"['Crime', 'Drama']",9.2,1972,R,175,70
2,3,The Dark Knight,"['Action', 'Crime', 'Drama']",9.0,2008,PG-13,152,00
3,4,The Godfather Part II,"['Crime', 'Drama']",9.0,1974,R,202,70
4,5,12 Angry Men,"['Crime', 'Drama']",9.0,1957,Approved,96,50


In [33]:
# split genres into their own columns and strip the list elements from the string
genre_split = df['genre'].str.replace(r"[',\[\]]", "", regex=True).str.split(' ', expand=True)

genre_split = genre_split.rename(columns={0: 'genre_1', 1: 'genre_2', 2: 'genre_3'})

In [34]:
df = pd.concat([df, genre_split], axis=1)

df.head()

,rank,title,genre,imdb_rating,year,rating,run_time,decade,genre_1,genre_2,genre_3
0,1,The Shawshank Redemption,['Drama'],9.3,1994,R,142,90,Drama,None,None
1,2,The Godfather,"['Crime', 'Drama']",9.2,1972,R,175,70,Crime,Drama,None
2,3,The Dark Knight,"['Action', 'Crime', 'Drama']",9.0,2008,PG-13,152,00,Action,Crime,Drama
3,4,The Godfather Part II,"['Crime', 'Drama']",9.0,1974,R,202,70,Crime,Drama,None
4,5,12 Angry Men,"['Crime', 'Drama']",9.0,1957,Approved,96,50,Crime,Drama,None


In [35]:
# organize columns to get decade after year and genres 
df = df[['rank', 'title', 'genre_1', 'genre_2', 'genre_3', 'imdb_rating', 'year', 'decade', 'rating', 'run_time']]

df

,rank,title,genre_1,genre_2,genre_3,imdb_rating,year,decade,rating,run_time
0,1,The Shawshank Redemption,Drama,None,None,9.3,1994,90,R,142
1,2,The Godfather,Crime,Drama,None,9.2,1972,70,R,175
2,3,The Dark Knight,Action,Crime,Drama,9.0,2008,00,PG-13,152
3,4,The Godfather Part II,Crime,Drama,None,9.0,1974,70,R,202
4,5,12 Angry Men,Crime,Drama,None,9.0,1957,50,Approved,96
...,...,...,...,...,...,...,...,...,...,...
95,96,Reservoir Dogs,Crime,Thriller,None,8.3,1992,90,R,99
96,97,Ikiru,Drama,None,None,8.3,1952,50,Not Rated,143
97,98,Lawrence of Arabia,Adventure,Biography,Drama,8.3,1962,60,PG,218
98,99,Citizen Kane,Drama,Mystery,None,8.3,1941,40,PG,119


In [36]:
df.to_csv('../data/processed/top_100_movies_processed.csv', index=False, header=True)

## Data Analysis

In [37]:
df = pd.read_csv('../data/processed/top_100_movies_processed.csv')

df.head()

,rank,title,genre_1,genre_2,genre_3,imdb_rating,year,decade,rating,run_time
0,1,The Shawshank Redemption,Drama,NaN,NaN,9.3,1994,90,R,142
1,2,The Godfather,Crime,Drama,NaN,9.2,1972,70,R,175
2,3,The Dark Knight,Action,Crime,Drama,9.0,2008,0,PG-13,152
3,4,The Godfather Part II,Crime,Drama,NaN,9.0,1974,70,R,202
4,5,12 Angry Men,Crime,Drama,NaN,9.0,1957,50,Approved,96


In [51]:
unique_genres = set()

genre_1 = df['genre_1'].value_counts().index

genre_2 = df['genre_2'].value_counts().index

genre_3 = df['genre_3'].value_counts().index

unique_genres.update(genre_1, genre_2, genre_3)

unique_genres = sorted(list(unique_genres))

unique_genres

['Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']